In [96]:
#Import required libraries
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [97]:
#Import project specific functions
#sys.path.append('..')
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [98]:
#Import shared functions
sys.path.append('..')
#sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *
from IPM_Shared_Code_public.Python.google_functions import *

### Use the config file to setup connections

In [99]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [100]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [101]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('patrol_dpr')

In [102]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [103]:
cols = list(col_rename.values())

### Read the current data from SQL

In [104]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_patrol'

In [105]:
patrol_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['patrol_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [106]:
if patrol_sql.shape()[0] > 0:
    format_datetime(patrol_sql, 'encounter_timestamp')
    format_datetime(patrol_sql, 'encounter_datetime')

In [107]:
sql_cols = list(patrol_sql.columns.values)

In [108]:
float_cols = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
              'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']
for c in float_cols:
    patrol_sql[c] = patrol_sql[c].astype(float)

In [109]:
patrol_sql.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough,closed_outcome_spec,sd_outcome_spec
0,06-01-2020 11:29:57,06-01-2020 11:12:00,R016,cedar grove beach,Rangers,Standard Patrol,Emily,Nacchio,Glen,Higgins,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,NaN,NaN
1,06-01-2020 11:28:49,06-01-2020 11:27:00,R110,playground,Rangers,Standard Patrol,anthony,denicola,Nicholas,wyant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,NaN,NaN
2,06-01-2020 11:26:41,06-01-2020 11:30:00,NaN,NaN,PEP,Standard Patrol,SGT IRIS,Hernandez,Richard,Diaz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens,NaN,NaN
3,06-01-2020 11:26:28,06-01-2020 11:17:00,X010A,NaN,PEP,Standard Patrol,Tezra,Swinea,Oscar,Marrero,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,NaN,NaN
4,06-01-2020 11:24:54,06-01-2020 10:56:00,R019,NaN,PEP,Standard Patrol,J,Krueger,S,Villegas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island,NaN,NaN


In [110]:
hash_rows(patrol_sql, exclude_cols = ['site_id', 'encounter_timestamp', 'encounter_datetime'], hash_name = 'row_hash')

### Read the site reference list from SQL

In [111]:
sql = '''select *
         from (select *,
                      row_number() over(partition by site_desc, park_borough order by site_desc) as n
               from crowdsdb.dbo.tbl_ref_park_sites
               where active = 1) as t
         where n = 1'''

In [112]:
site_ref = (pd.read_sql(con = engine, sql = sql)
            .rename(columns = {'park_borough': 'borough'}))[['site_id', 'site_desc', 'borough']]

### Read the latest data from Google Sheets

In [113]:
#Read the worksheet as a data frame, rename the columns and subset the columns to only include those
#in the column list
patrol = (read_google_sheet(cred_file, '_COMBINED Patrol Reporting Responses', 'MASTER', drop_empty_cols = False,
                            evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [114]:
format_datetime(patrol, 'encounter_timestamp')
format_datetime(patrol, 'encounter_datetime')

In [115]:
patrol.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_comments,sd_pdassist,sd_pdcontact,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
1,06-30-2020 15:46:29,NaT,Franz Sigel Park,Dog Run,PEP,Tezra,Swinea,Reynell,De Los Santos,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,06-30-2020 15:44:47,NaT,Franz Sigel Park,NaN,PEP,Tezra,Swinea,Reynell,De Los Santos,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,06-30-2020 15:42:55,NaT,Macombs Dam Park,NaN,PEP,Tezra,Swinea,Reynell,De Los Santos,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,06-30-2020 15:42:29,06-30-2020 15:40:00,Grover Cleveland Playground,NaN,PEP,walter,Luciano,Annette,Probst,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
5,06-30-2020 15:41:23,NaT,River Avenue Skate Park,NaN,PEP,Tezra,Swinea,Reynell,De Los Santos,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx


In [116]:
yesno = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
         'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']

In [117]:
yesno_cols(patrol, yesno)

In [118]:
#Remove any rows with no data, presumably these are rows with no timestamp
patrol = patrol[patrol['encounter_timestamp'].notna()]

In [119]:
patrol = patrol.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [28]:
hash_rows(patrol, exclude_cols = ['site_id', 'encounter_timestamp', 'encounter_datetime'], hash_name = 'row_hash')

In [29]:
patrol_deltas = (check_deltas(new_df = patrol, old_df = patrol_sql, on = ['site_id', 'encounter_timestamp', 'encounter_datetime'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [30]:
patrol_inserts = patrol_deltas[patrol_deltas['dml_verb'] == 'I'][sql_cols]

In [31]:
patrol_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
0,06-15-2020 08:51:45,06-15-2020 08:50:00,M206,NaN,PEP,Standard Patrol,B,Kanhai,K,Rodriguez,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
1,06-15-2020 08:45:55,06-15-2020 08:44:00,R006,NaN,PEP,Standard Patrol,Ann Marie,Santos,Anthony,Twomley,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
2,06-15-2020 08:42:40,06-15-2020 08:30:00,X034,Skate park,PEP,Standard Patrol,Reynell,DELOSSANTOS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,06-15-2020 08:25:21,06-15-2020 08:23:00,M108Q1,NaN,PEP,Standard Patrol,1323,ss,1432,rm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,06-15-2020 08:20:59,06-15-2020 08:15:00,X002,Bronx Park East/Bronxdale Ave,PEP,Standard Patrol,Zechariah,Frederickson,Jerry,cuello,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx


In [32]:
patrol_inserts.to_sql('tbl_dpr_patrol', engine, index = False, if_exists = 'append')

In [33]:
patrol_updates = patrol_deltas[patrol_deltas['dml_verb'] == 'U'][sql_cols]

In [34]:
patrol_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
922,06-12-2020 19:11:43,06-12-2020 19:05:00,X255,NaN,PEP,Standard Patrol,Sorie,Kamara,Naji,Williams,...,200.0,200.0,Bench/sitting area,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
926,06-12-2020 18:45:18,06-12-2020 18:25:00,X045,The baseball field gate by the bathroom is dam...,PEP,Standard Patrol,Sorie,Kamara,Naji,Williams,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
957,06-12-2020 17:26:45,06-12-2020 16:47:00,X147A,NaN,PEP,Complaint,IVAN,Bernardez,Kevin,Feliciano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
958,06-12-2020 17:25:16,06-12-2020 17:20:00,X030,NaN,PEP,Standard Patrol,sorie,Kamara,Naji,Williams,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
960,06-12-2020 17:22:43,06-12-2020 16:30:00,M376,NaN,PEP,Fixed Post,Anna,Williams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [35]:
sql_update(patrol_updates, 'tbl_dpr_patrol', engine, ['encounter_timestamp', 'encounter_datetime', 'site_id'])